In [ ]:
#pip install sounddevice scipy openai-whisper bark

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper
from bark import generate_audio, save_audio
import numpy as np
import os
import wave

def record_audio(filename="input_audio.wav", duration=5, samplerate=16000):
    print("Listening... Speak now!")
    audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("Recording finished.")
    write(filename, samplerate, audio_data)  # Save as WAV file
    return filename

def transcribe_audio(filename):
    model = whisper.load_model("small")  # Change to "base", "small", "medium", or "large" if needed
    print("Transcribing...")
    result = model.transcribe(filename)
    return result["text"]

def speak_text(text, output_file="output_audio.wav"):
    print("Generating response...")
    audio_array = generate_audio(text)
    save_audio(audio_array, output_file)
    print(f"Response saved as {output_file}.")
    return output_file

def play_audio(file_path):
    with wave.open(file_path, 'rb') as wf:
        rate = wf.getframerate()
        data = wf.readframes(wf.getnframes())
        audio_data = np.frombuffer(data, dtype=np.int16)
        sd.play(audio_data, samplerate=rate)
        sd.wait()

def conversational_assistant():
    print("Welcome to your fully local conversational assistant!")
    while True:
        # Step 1: Record user input
        input_audio = record_audio(duration=5)
        user_input = transcribe_audio(input_audio)
        print(f"User said: {user_input}")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Goodbye!")
            break

        response_text = f"You just said: {user_input}. How can I assist you further?"
        response_audio = speak_text(response_text)
        play_audio(response_audio)

# Run the assistant
if __name__ == "__main__":
    conversational_assistant()
